# Módulo 1: Análisis de datos en el ecosistema Python

### Sesión (13)

**12/12/2022**

## Análisis de calidad del aire


En los últimos años, los **altos niveles de contaminación** durante ciertos periodos secos en **Madrid** ha obligado a las autoridades a tomar medidas contra el uso de automóviles en el centro de la ciudad, y ha sido utilizado como razón para **proponer modificaciones drásticas en el urbanismo de la ciudad**. 

Gracias a la **web de [Datos Abiertos del Ayuntamiento de Madrid](https://datos.madrid.es/portal/site/egob/menuitem.400a817358ce98c34e937436a8a409a0/?vgnextoid=eba412b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextchannel=eba412b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default)**, los datos de calidad del aire están públicamente disponibles e incluyen **datos históricos diarios y horarios de los niveles registrados desde 2001 hasta 2018** y la lista de estaciones que se utilizan para el análisis de contaminación.

Vamos a utilizar **una muestra** que se ha preparado en base a estos datos que muestran la **calidad del aire** en varias estaciones de **Madrid** según diferentes variables.

In [ ]:
# importamos las librerías necesarias 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
# Modificamos los parámetros de los gráficos en matplotlib
from matplotlib.pyplot import rcParams

rcParams['figure.figsize'] = 12, 6 # el primer dígito es el ancho y el segundo el alto
rcParams["font.weight"] = "bold"
rcParams["font.size"] = 10
rcParams["axes.labelweight"] = "bold"

### Cargar los datos para el modelo

In [ ]:
# Cargar el dataset desde un archivo

df_aire = pd.read_excel("ABT_CALIDAD_AIRE.xlsx")

In [ ]:
# Consultar los registros del DataFrame
df_aire

In [ ]:
# Conteo de valores perdidos/faltantes  
df_aire.isna().sum()

In [ ]:
# La información útil sobre los datos guardados en formato DataFrame
df_aire.info()

In [ ]:
# Consultar los valores nulos
df_aire['Intensidad_Pto_trafico1'].isna()

In [ ]:
# Contar los valores nulos para este campo
df_aire['Intensidad_Pto_trafico1'].isna().sum()

In [ ]:
# Mostrar los valores nulos para este campo
df_aire['Intensidad_Pto_trafico1'][df_aire['Intensidad_Pto_trafico1'].isna()]

In [ ]:
# Sacar los registros que contienen valores nulos para este campo
df_aire[67801:67901][['Intensidad_Pto_trafico1', 'nombre_estacion', 'anyo','mes', 'dia', 'hora']]

In [ ]:
# Consultamos los registros que tienen algún valor nulo
df_aire.drop(df_aire.dropna().index)

In [ ]:
# El tamaño esperado para el dataset limpio
df_aire.shape[0] - 20367 

Para evitar problemas posteriores, usamos el método ``dropna()`` para limpiar el tablón de valores perdidos (**missing values**) y reiniciamos el índice. Comprobamos la cantidad de las filas filtradas.

In [ ]:
df_air_filt = df_aire.dropna().reset_index(drop=True)
df_air_filt

### Análisis Exploratorio Inicial, Tratamiento y Limpieza de datos

In [ ]:
# Echamos un vistazo a las características de cada columna
df_air_filt.describe()

In [ ]:
# Consultamos el tipo de datos
df_air_filt.dtypes

Los algoritmos **entienden de números y no otra cosa!**, con lo cual para que el algoritmo pueda trabajar con toda la información del dataset, los datos se tienen que **transformar en valores numéricos**.

In [ ]:
# Consultar las variables que son del tipo "string"
df_air_filt.dtypes[df_air_filt.dtypes=='object']

Salvo la variable `wind_dir` que indica un parámetro meteorológico y lo tendrémos que tratar como una variable categórica más adelante, el resto se pueden quitar por no ser tan relevante.

In [ ]:
# Analizar las direcciones del tiempo
df_air_filt['wind_dir'].value_counts()

Se observa que en el conjunto de datos existen datos **no tan descriptivos** como el **nombre de la estación** y alguna información asociada a estas estaciones como puede ser el **año** o el **tipo de estación**. 

In [ ]:
# Datos informativos
df_air_filt['id_pto_calidad'].value_counts()

In [ ]:
# Datos informativos
df_air_filt['Pto_trafico4'].value_counts()

En la lista ``columnas_drop`` definimos las columnas a eliminar del dataset por ser meramente informativas.

In [ ]:
columnas_drop= ["Pto_trafico1",
                "Pto_trafico2",
                "Pto_trafico3",
                "Pto_trafico4",
                "Pto_trafico5",
                "Pto_trafico5",
                "anyo",
                "id_pto_calidad",
                "nombre_estacion",
                "tipo_estacion_id",
                "fecha"]

Creamos un nuevo dataset que sea como el tablón anterior, eliminando las columnas de la lista indicada. Utilizamos el método `drop` para _DataFrames_ de _pandas_.

In [ ]:
df_air_filt2 = df_air_filt.drop(columns=columnas_drop)
print("Tamaño del tablón filtrado: ", df_air_filt.shape)
print("Tamaño del tablón nuevo: ", df_air_filt2.shape)

In [ ]:
# Tipología de las variables exsitentes en el nuevo dataset
df_air_filt2.dtypes.value_counts()

Vamos a analizar el caso de la única variable todavía no-numérica (`wind_dir`) que necesita un tratamiento concreto. _sklearn_ posee directamente métodos para convertir estas variables en numéricas. Así, tenemos:  

* ``sklearn.preprocessing.LabelEncoder``: Recibe un array de strings o enteros y nos devuelve uno de enteros con valores comprendidos **entre _0_ y _n-1_**, donde **_n_ es el número total de categorías** de la variable.  

* El problema de _LabelEncoder_ es que para más de dos clases, el algoritmo podrá entender que seguimos teniendo una relación de orden entre los datos, es decir: Una variable categórica que por ejemplo recoja puntos cardinales (N,S,E,O) indica diferentes valores pero el norte no es de mayor ni menor importancia que el oeste. Simplemente, es distinto. Si aplicamos el método _LabelEncoder_ nos devolverá un array con valores entre (0,1,2,3). En algunos casos y **algunos algoritmos pueden deducir que en estos datos existe una relación de orden**, lo cual no es cierto.  

* Para evitar este problema, se recurre a ``sklearn.preprocessing.OneHotEncoder``. Este **genera _n-1_ variables "dummies" o binarias**, es decir, que toman valores (0,1). Aquí ya se evita el que pueda inferirse un orden en las categorías, pues aquí sí que la variable significa _1== "es norte"_ y _0 =="no es norte"_ y así sucesivamente.  
El problema que se presenta en este caso, es que **si tenemos muchas variables categóricas con muchas tipologías y clases, al convertirlas en binarias, se nos aumenta notablemente el tamaño del dataset**.

En nuestro dataset hemos visto que hay una variable llamada **"wind_dir"** que indica la dirección del viento. Esta variable tiene 16 categorías:  
- 'NE'  
- 'ENE'  
- 'E'
- 'ESE'
- 'SE'
- 'SSE'
- 'S'
- 'SSW'
- 'SW'
- 'WSW'
- 'W'
- 'NNE'
- 'N'
- 'WNW'
- 'NW'
- 'NNW'

Primero vamos a **agruparlas en cuatro grupos de (N,S,E,O)** y después convertirlas en valres numéricos.

In [ ]:
# La variable de dirección de viento que se requiere agrupar y posteriormente convertirse en números
df_air_filt2['wind_dir'].value_counts()

In [ ]:
# Utilizamos estas listas como categorías para poder agruparlos posteriormente
norte = ['NNE','NNW','NE','N']
sur = ['SSE','SSW','SW','S']
este = ['ENE','ESE','SE','E']
oeste = ['WNW','WSW','NW','W']

In [ ]:
# Creamos una nueva columna en una nueva DataFrame con los datos agrupados
df_air_filt3 = df_air_filt2.copy()
df_air_filt3['wind_dir'] = df_air_filt3['wind_dir'].apply(lambda x: "N" if x in norte else
                                                                    "S" if x in sur else
                                                                    "O" if x in oeste else "E")

In [ ]:
# Consultamos los nuevos valores reemplazados
df_air_filt3['wind_dir'].value_counts()

In [ ]:
# Comprobamos que están todas las celdas bien agrupadas
df_air_filt3.groupby('wind_dir').count()['mes']

Una vez que tenemos ya la columna actualizada, vamos a utilizar los __encoders__ de _sklearn_ para transformarlos. Para no añadir más columnas al dataset procedemos con el método _LabelEncoder_:
* La función ``sklearn.preprocessing.LabelEncoder`` codifica las etiquetas de una variable categórica en valores numéricos **entre 0 y el número de clases menos 1**.  

* Una vez instanciado el encoder, el método ``fit`` lo entrena, **creando el mapeado entre las etiquetas y los números** según las distintas categorías presentes en dicha variable.   

* El método ``transform`` asigna para cada etiqueta los números correspondientes, **aplicando el mapeado** creado en el paso anterior.  

* El método ``fit_transform`` realiza **ambas acciones conjuntamente**.  

In [ ]:
from sklearn.preprocessing import LabelEncoder
etiquetado = LabelEncoder()
etiquetado.fit(df_air_filt3['wind_dir'])

Un atributo de esta función llamado ``classes_`` almacena **el array que mapea las etiquetas** y asigna los números según el índice de cada etiqueta en el array.

In [ ]:
# Obtenemos las propiedades:
etiquetado.classes_

In [ ]:
# Transformamos el dataset aplicando el mapeado:
etiquetado.transform(df_air_filt3['wind_dir'])

Se ve que al aplicar el mapeado con el método _transform_ obtenemos un array de numpy de tipo enteros. Vamos a añadir en **una nueva columna** llamada **'Dir_viento_etiquetado'** con estos valores númericos y consultamos si existe una relación entre la columna `wind_dir` y esta nueva columna de `Dir_viento_etiquetado` antes de borrar los datos no-numéricos.


In [ ]:
# Asignar una nueva columna
df_air_filt3['Dir_viento_etiquetado'] = etiquetado.transform(df_air_filt3['wind_dir'])

# mostrar la relación con el dato original
df_air_filt3.groupby(['wind_dir','Dir_viento_etiquetado']).count()[['mes']]

In [ ]:
# Definimos un nuevo dataframe con solamente datos transformados a valores numéricos
df_air_filt4 = df_air_filt3.drop('wind_dir', axis='columns')

# Consultar el tipo de las variables
df_air_filt4.dtypes.value_counts()

### **Reducción de Variables (_Dimensionality Reduction_)**

Existen muchos métodos de reducción de variables existentes en _sklearn_.

- #### **Filtro por varianza**: 

Se define un umbral de varianza usando ``from sklearn.feature_selection import VarianceThreshold`` y todas las variables que no lo cumplan se eliminan. **Muy útil para eliminar variables que son casi constantes**. Dispone de los métodos`` fit`` y ``transform`` para aplicar a un dataset. 

El problema es que devuelve un array con las variables no eliminadas y puede que no resulte sencillo rastrear cuáles ha eliminado. Normalmente hay que ir comparando columna a columna en el dataframe original y el array para ver cuáles ha eliminado y cuáles no.


- #### **Filtros univariantes basados en una clasificación de p-values**.

Según el modelo sea de clasificación o de regresión, se aplica un **test estadístico (_chi cuadrado_, _anova_ respectivametne)** y tras indicar con cuántas variables queremos quedarnos, se crea una clasificación de variables y el modelo selecciona las _k_ con los _p-valores_ menores con mayor grado de independencia entre variables.


- #### **Selección basada en árbol de decisión (_Decision Trees Importances_)**.

Consiste en **entrenar un árbol de decisión muy sobreajustado** sobre todo el dataset y después quedarse con las variables que expliquen un valor determinado de la información: 90%, 95 %....

Este método utiliza un modelo y como veremos más adelante, todos los modelos de sklearn tienen los siguientes métodos:  
  - ``.fit(X=conjunto de train de variables independientes, y=variable objetivo del conjunto de train)``  
  - ``.predict(X=conjunto de variables independientes)``. Siempre tiene que tener las mismas variables que el que se utilizó para el .fit()  
  - ``.score(y_real, y_predicción)`` Devuelve, **para el caso de regresión el $R^2$** del modelo y **para clasificación el accuracy** entendida como el porcentaje de aciertos sobre el total.

- #### **Selección basada en métodos recursivos**.
Este caso funciona de modo similar a como lo hacen las regresiones "_backward_", es decir, se comienza probando todas las variables para ir sacando variables una a una. 

### Importancia de variables

Definimos el conjunto de las variables de entrada (_variables independientes_) y la variable objetivo (`Calidad_NO2`), y almacenamos esta última en una variable llamada `target`.

Importamos desde la librería _sklearn_ la clase para el _árbol de regresión_. Y procedemos a entrenar uno con todo el dataset y así obtener las variables más importantes.

In [ ]:
# Generar la lista de features y la variable target
target = 'Calidad_NO2'
features = [x for x in df_air_filt4.columns if x!=target]

print(target)
print(features)

In [ ]:
# Consultar la variable de salida
df_air_filt4[target]

In [ ]:
# Importar el algoritmo de árboles de decisión
from sklearn.tree import DecisionTreeRegressor

# Asignar el algortimo e indicar la profundidad máxima del árbol (con un número rotandamente grande para sobreajustar)
arbol_importancia = DecisionTreeRegressor(max_depth=len(features)+10, random_state=100)

# Entrenar un árbol con todo el conjunto de datos
arbol_importancia.fit(X=df_air_filt4[features], y=df_air_filt4[target])

In [ ]:
# Comprobamos que se obtiene un R^2 muy alto. Lo desesable es que sea 1.
y_pred_arbol = arbol_importancia.predict(X=df_air_filt4[features])

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# Métricas para evaluar la calidad del modelo
print('Mean Absolute Error:', mean_absolute_error(df_air_filt4[target], y_pred_arbol))
print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(df_air_filt4[target], y_pred_arbol)*100)
print('Mean Squared Error:', mean_squared_error(df_air_filt4[target], y_pred_arbol))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(df_air_filt4[target], y_pred_arbol)))
print('R^2 coefficient of determination:', r2_score(df_air_filt4[target], y_pred_arbol))

El modelo consigue un rendimiento perfecto!! Ahora vamos a sacar **las variables más importantes** que ha detectado el modelo. El **árbol devuelve la importancia de cada una de ellas** en el orden en el que están las columnas en el dataframe. 

In [ ]:
# Importancias de cada variable en el árbol ajustado (Gini importance)
arbol_importancia.feature_importances_

Se observa que el método ``feature_importances_`` devuelve un array con la importancia en tanto por uno. Para asociarlos, podemos crear una serie con los índices de las variables y al lado la importancia.

In [ ]:
# Creamos un DataFrame con los datos de importancia
importancia = pd.DataFrame(arbol_importancia.feature_importances_, index=features, columns=["Importancia"])

# Ordenamos los datos
importancia.sort_values(by=importancia.columns[0], ascending=False, inplace=True)
importancia

Podemos obtener una lista con **las variables que aglutinan el 85% de la información**. Procedemos a añadir al DataFrame "_importancia_" que hemos creado en el caso anterior **la columna `imp_acum` que acumule la suma**.

In [ ]:
importancia["imp_acum"] = importancia["Importancia"].cumsum()
importancia

In [ ]:
# Conjunto de variables más importantes
importancia.loc[importancia['imp_acum']<=0.85]

In [ ]:
# Definimos la lista de variables no tan importantes, cortando por el porcentaje de 85% de la información necesaria
variables = importancia.loc[importancia['imp_acum']>0.85].index.to_list()
print(variables)

In [ ]:
print("Variables iniciales: ", len(features))
print("Variables no importantes (a eliminar): ", len(variables))

Ahora podemos filtrar el tablón para quedarnos solamente con las variables más importantes.

In [ ]:
df_air_filt5 = df_air_filt4.drop(labels=variables, axis='columns')
df_air_filt5

### **Planteamiento del ejercicio de clasificación**

Vamos a crear una variable objetivo de nombre ***Escenario*** con 2 clases como 0,1 del modo que:  

* El **nivel 0** se corresponda a los valores **por debajo del percentil 33** de la variable _target_ (`Calidad_NO2`).    
* El **nivel 1** se corresponde a los valores **por encima del percentil 33**.  

Eliminamos después la columna `Calidad_NO2` del nuevo dataset y procedemos a dividir el datset en conjuntos de train y test (usualmente con un reparto de 80% - 20%).  

**Vamos a intentar predecir si la calidad de aire de las distintas zonas está en el nivel 0, 1.**


In [ ]:
# Hacemos una copia del tablón filtrado para no trabajar sobre original
df_aire_calidad = df_air_filt5.copy()

In [ ]:
# Crear la columna nueva "Escenario" y borrar la columna target antigua 'Calidad_NO2'
df_aire_calidad["Escenario"] = np.where(df_aire_calidad['Calidad_NO2']<df_aire_calidad['Calidad_NO2'].quantile(0.33),0,1)

df_aire_calidad.drop(['Calidad_NO2'], axis='columns', inplace=True)
df_aire_calidad

In [ ]:
df_aire_calidad.describe()

In [ ]:
# Graficar la distribución de los valores originales
plt.figure(figsize=(20,6))
sns.violinplot(data=df_aire_calidad, orient='v')
plt.show()

### **Paso 1.**  Obtención y preparación de datos

In [ ]:
## Preparar el conjunto de datos del modelo

# Variables independientes (features)
X = df_aire_calidad.drop('Escenario', axis='columns')

# Variable dependiente (target) que son los niveles de aire con menos contaminación
y = df_aire_calidad['Escenario']

Se puede observar la necesidad de llevar todas las variables de entrada a una escala estándar.

In [ ]:
# importar los objetos necesarios de la librería sklearn
from sklearn.preprocessing import StandardScaler

# declarar el tipo de escalamiento y aplicarlo al conjunto de datos
escalado = StandardScaler().fit(X)
dataset_normal = escalado.transform(X)
dataset_normal

In [ ]:
# Lo convertimos en un DataFrame, añadiendole sus etiquetas
X_normal = pd.DataFrame(dataset_normal, columns=X.columns)
print(type(X_normal))
X_normal

In [ ]:
X_normal.describe().round(4)

In [ ]:
# Graficar la distribución de los valores estandarizados
plt.figure(figsize=(20,6))
sns.boxplot(data=X_normal,  orient='v')
plt.show()

### **Paso 2.**  Dividir el dataset en Training y Test

In [ ]:
# Separar los conjuntos de datos de entrenamiento (Training) y de prueba (Test) para las variables de entrada y salida
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_normal, y, test_size=0.2, random_state=88)

In [ ]:
# "test_size" representa la proporción del conjunto de datos a incluir en la división de Test
print(X_train.shape[0])
print(X_test.shape[0])
X_train.shape[0] + X_test.shape[0]

### **Paso 3.** Cargar y elegir el modelo de regresión logística

In [ ]:
# Importar el módulo que corresponde al algoritmo
from sklearn.linear_model import LogisticRegression

# Asignar el algoritmo que vamos a aplicar 
log_r = LogisticRegression(max_iter=1000,
                           random_state=100)

### **Paso 4.** Entrenar el modelo de regresión logística con los datos de entrenamiento

In [ ]:
# Entrenar el modelo
log_r.fit(X_train, y_train)

Ahora que el modelo está entrenado, sacamos las predicciones, analizamos los resultados y obtenemos algunas métricas del modelo basadas en el conjunto de datos de prueba. Según las métricas, podremos observar si el modelo clasificó correctamente todas los niveles definidos de la calidad del aire.

### **Paso 5.** Obtener las predicciones 

In [ ]:
# Calcular las predicciones con el conjunto de prueba
y_pred = log_r.predict(X_test)

In [ ]:
# Imprimir la salida del modelo (los niveles de calidad del aire)
print(y_pred)

### **Paso 6.** Evaluación del modelo a través de sus métricas

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

Existen otra serie de metricas para calificar los modelos de clasificación que se detallan a continuación. Algunas de estas medidas se resumen en un informe llamado **classification_report**.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(y_test, y_pred),
                               display_labels=log_r.classes_)
disp.plot()

plt.show()

In [ ]:
# Calcular el área bajo la curva de funcionamiento del receptor
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

In [ ]:
# Graficar la curva ROC
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(y_test, y_pred)

plt.show()

---

### **`Ejercicio 13`**

Vamos a realizar un **planteamiento ligeramente distinto** para el mismo ejercicio desarrollado en esta sesión. Posteriormente analizamos el rendimiento del nuevo clasificador (todavía del tipo *logit*):

**`13.1`** __Features__: Define una nueva lista de variables no muy importantes y filtra las variables originales, tratadas y almacenadas en un paso anterior como __`df_air_filt4`__, ésta vez para quedarnos con el conjunto de características que componen el `95%` de la información necesaria para modelizar y estimar la variable objetivo.

**`13.2`** __Target__: Crea mediante un nuevo planteamiento una variable de salida que clasifique solamente la calidad del aire por debajo del primer cuartil (__Q1__).

**`13.3`** __Scaler__: Aplica un escalamiento diferente para llevar ésta vez a todos los datos **a una escala entre 0 y 1**. 

**`13.4`** Crea un nuevo detector usando el método de _regresión logística_ con el nuevo tratamiento y estos nuevos conjuntos de variables de entrada y de salida. Consulta todas las metricas y visualiza las gráficas que muestran el rendimiento del modelo resultante y explica si se puede elegir a este como un buen clasificador de calidad del aire de Madrid en comparación con el anterior modelo desarrollado en la sesión!!

In [ ]:
## Solución
# Ejercicio 13.1


In [ ]:
## Solución
# Ejercicio 13.2


In [ ]:
## Solución
# Ejercicio 13.3
